In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [ ]:
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]

df_daily, feature_sets, return_cols, daily_cols, sets, set_names, feature_cols = train_flow.import_data(ticker, include_minute_feats, returns) 

# Add any new features
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
df_main = df_daily[df_daily['Date'] <= '2026-02-06'].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


# Run Train_Flow

In [13]:
# -----------------------------
# Handling of new features and permutation importance
# -----------------------------
pi_handlings = ['include_new'] #'exclude_new', 'include_new', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
new_features = [] #past_sum_cols

# -----------------------------
# Run grid (feature sets x horizon x train_years, etc.)
# -----------------------------
target_horizons = [2, 10, 30]#[2, 5, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 1
test_day = 1
feature_cols = feature_cols
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
list_name = 'kitchen_sink'

results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)

Running for horizon 2 | include_new
181 | 8 | All Cols: ['10_SMA_50', 'SMA_10_Lag100_max', 'SMA_10_Lag100_min', 'SMA_10_Lag25_max', 'SMA_10_Lag25_min', 'SMA_10_Lag50_max', 'SMA_10_Lag50_min', 'Vol_Ratio_50_zscore']
Run 1/1 | Train: 2021-03-17 → 2026-01-30 | Test: 2026-02-04 → 2026-02-04 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
Running for horizon 10 | include_new
181 | 8 | All Cols: ['10_SMA_50', 'SMA_10_Lag100_max', 'SMA_10_Lag100_min', 'SMA_10_Lag25_max', 'SMA_10_Lag25_min', 'SMA_10_Lag50_max', 'SMA_10_Lag50_min', 'num_days_200']
Run 1/1 | Train: 2021-02-23 → 2026-01-07 | Test: 2026-01-23 → 2026-01-23 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
Running for horizon 30 | include_new
181 | 8 | All Cols: ['Min_240_Rows_Since', 'SMA_100_Lag100_max', 'SMA_100_Lag200_min', 'SMA_10_Lag200_min', 'SMA_10_Lag25_max', 'SMA_10_Lag25_min', 'SMA_25_Lag150_max', 'num_days_200']
Run 1/1 | Train: 2020-12-23 → 2025-11-07 | Test: 2025-12-23 → 2025-12-23 | Train_n=1225 | Test_n=1

# Save most recent run

In [ ]:
df = pd.read_csv("baseline_all.csv")
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
        'pi_size', 'min_feats']
df_new = results_df[cols].copy()
df_concat = pd.concat([df[cols], df_new], ignore_index=True)
df_concat.to_csv('baseline_new.csv', index=False)

# Run Performance_Flow

In [10]:
import performance_flow
importlib.reload(performance_flow)

results_file_name = "horizon_5_baseline.csv"
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)

min_th = 0.55
cov_th = 0.75
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)